#### Prerequisite

In [ ]:
!python -m pip install --upgrade pip

#### Create a docker image

This step can run in SageMaker classic notebook environment or in your own local environment with docker installed.

In [1]:
# Image name below preferably starts with the prefix `sagemaker`
!docker build -t sagemaker_feature_encoder -f Dockerfile .

Sending build context to Docker daemon  91.14kB
Step 1/7 : FROM python:3.7
 ---> 5cda39795abb
Step 2/7 : COPY requirements.txt ./
 ---> Using cache
 ---> d54ca5598bf4
Step 3/7 : RUN python -m pip install --upgrade pip
 ---> Using cache
 ---> 6b5d5e0c3e3f
Step 4/7 : RUN pip install --no-cache-dir -r requirements.txt
 ---> Using cache
 ---> 9bac79560c84
Step 5/7 : COPY serve /usr/local/bin
 ---> Using cache
 ---> 1e5ed6c813f0
Step 6/7 : RUN chmod +x /usr/local/bin/serve
 ---> Using cache
 ---> 166f0caad908
Step 7/7 : EXPOSE 8080
 ---> Using cache
 ---> 6564c8a85bf1
Successfully built 6564c8a85bf1
Successfully tagged sagemaker_feature_encoder:latest


#### Push docker image from local to ECR

In [2]:
%%sh

# Specify a name to your custom container
container_name=sagemaker_feature_encoder  # should match name from previous cell
echo "Container Name: " ${container_name}

# Retreive AWS account ID
account=$(aws sts get-caller-identity --query Account --output text)

# Get the AWS region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo "Account: " ${account}
echo "Region: "${region}

repository="${account}.dkr.ecr.${region}.amazonaws.com"
echo "ECR Repository: " ${repository}

image="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:latest"
echo "ECR Image URI: " ${image}

# If the ECR repository does not exist, create it.
aws ecr describe-repositories --repository-names ${container_name} > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name ${container_name} > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${repository}

# Tag the local image with ECR image name
docker tag ${container_name} ${image}

# Finally, push the local docker image to ECR with the full ECR image name
docker push ${image}

Container Name:  sagemaker_feature_encoder
Account:  892313895307
Region: us-east-1
ECR Repository:  892313895307.dkr.ecr.us-east-1.amazonaws.com
ECR Image URI:  892313895307.dkr.ecr.us-east-1.amazonaws.com/sagemaker_feature_encoder:latest
Login Succeeded
The push refers to repository [892313895307.dkr.ecr.us-east-1.amazonaws.com/sagemaker_feature_encoder]
758c7aa4aa56: Preparing
e02f76465e84: Preparing
56b43070d8da: Preparing
2ec88b0c27f0: Preparing
58569f4de894: Preparing
eef135e35b6e: Preparing
a0db21004f62: Preparing
0b53caaeb40b: Preparing
1cad4dc57058: Preparing
4ff8844d474a: Preparing
b77487480ddb: Preparing
cd247c0fb37b: Preparing
cfdd5c3bd77e: Preparing
870a241bfebd: Preparing
eef135e35b6e: Waiting
a0db21004f62: Waiting
cd247c0fb37b: Waiting
870a241bfebd: Waiting
1cad4dc57058: Waiting
b77487480ddb: Waiting
0b53caaeb40b: Waiting
cfdd5c3bd77e: Waiting
4ff8844d474a: Waiting
2ec88b0c27f0: Layer already exists
e02f76465e84: Layer already exists
758c7aa4aa56: Layer already exists
58

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



##### Uncomment and run the command below if your local images need to be cleaned.

In [3]:
#!docker rmi $(docker images -q) -f

## Deploy feature encoder as a SageMaker real time endpoint

#### Imports 

In [4]:
from time import gmtime, strftime
import sagemaker
import datetime
import boto3
import time

#### Essentials

In [5]:
role = sagemaker.get_execution_role()
session = sagemaker.Session()
account = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=region)
image_name = 'sagemaker_feature_encoder'
image_uri = f'{account}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest'
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

#### Pack all encoders into a tar file and push to S3

In [6]:
!tar -czf encoder.tar.gz -C ./data .

In [7]:
default_bucket = session.default_bucket()
default_bucket

'sagemaker-us-east-1-892313895307'

In [8]:
!aws s3 cp encoder.tar.gz s3://{default_bucket}/

upload: ./encoder.tar.gz to s3://sagemaker-us-east-1-892313895307/encoder.tar.gz


#### Create a SageMaker model object

In [9]:
model_name = f'feature-encoder-{current_timestamp}'
model_artifacts_location = f's3://{default_bucket}/encoder.tar.gz'

In [10]:
response = sagemaker_client.create_model(ModelName=model_name, 
                                         Containers=[{
                                             'Image': image_uri, 
                                             'Mode': 'SingleModel', 
                                             'ModelDataUrl': model_artifacts_location}], 
                                         ExecutionRoleArn=role)
response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:892313895307:model/feature-encoder-2022-12-15-20-39-41',
 'ResponseMetadata': {'RequestId': '48fa04a8-c844-4e65-9126-f54d5d1ba5d5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '48fa04a8-c844-4e65-9126-f54d5d1ba5d5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Thu, 15 Dec 2022 20:39:44 GMT'},
  'RetryAttempts': 0}}

#### Create a SageMaker endpoint configuration

In [11]:
endpoint_config_name = f'feature-encoder-{current_timestamp}'

In [12]:
response = sagemaker_client.create_endpoint_config(EndpointConfigName=endpoint_config_name, 
                                                   ProductionVariants=[{
                                                       'VariantName': 'v1', 
                                                       'ModelName': model_name, 
                                                       'InstanceType': 'ml.c5.xlarge', 
                                                       'InitialInstanceCount': 2, 
                                                       'InitialVariantWeight': 1
                                                   }])
response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:892313895307:endpoint-config/feature-encoder-2022-12-15-20-39-41',
 'ResponseMetadata': {'RequestId': 'ee58184c-c0fa-4a13-9c0f-c011cd5deb32',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ee58184c-c0fa-4a13-9c0f-c011cd5deb32',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Thu, 15 Dec 2022 20:39:45 GMT'},
  'RetryAttempts': 0}}

#### Create a SageMaker endpoint

In [13]:
endpoint_name = f'feature-encoder-{current_timestamp}'

In [14]:
response = sagemaker_client.create_endpoint(EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)
response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:892313895307:endpoint/feature-encoder-2022-12-15-20-39-41',
 'ResponseMetadata': {'RequestId': '1efe686e-7624-4969-9a8c-31528f8a91b7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1efe686e-7624-4969-9a8c-31528f8a91b7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Thu, 15 Dec 2022 20:39:47 GMT'},
  'RetryAttempts': 0}}

#### Describe endpoint to track creation status

In [15]:
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response['EndpointStatus']

while endpoint_status == 'Creating':
    time.sleep(15)
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response['EndpointStatus'] 
    print(endpoint_status)

Creating
Creating
Creating
Creating
Creating
Creating
InService


#### Invoke endpoint to test deployed feature encoder

In [16]:
sagemaker_runtime = boto3.client('sagemaker-runtime', 
                                 region_name=region)

In [17]:
raw_payload = b"10-19-2021,145,24,22,14,usa,65+"

In [18]:
%%time

response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                             Body=raw_payload, 
                                             ContentType='text/csv')
response

CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 93.9 ms


{'ResponseMetadata': {'RequestId': '8d5365bf-1377-4611-98bc-c878a3db65d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8d5365bf-1377-4611-98bc-c878a3db65d8',
   'x-amzn-invoked-production-variant': 'v1',
   'date': 'Thu, 15 Dec 2022 20:41:41 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '128'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'v1',
 'Body': <botocore.response.StreamingBody at 0x7f14330555b0>}

##### Extract feature vector from the response

In [19]:
feature_vector = response['Body'].read().decode('utf-8').strip()
feature_vector

'0.04209896249002394,0.41690962099125367,0.3142857142857143,0.25925925925925924,0.208955223880597,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0'